# Ripple Carry Adder Workbok
**What is this workbook?** 
A workbook is a collection of problems, accompanied by solutions to them. The explanations focus on the logical steps required to solve a problem; they illustrate the concepts that need to be applied to come up with a solution to the problem, explaining the mathematical steps required.

Note that a workbook should not be the primary source of knowledge on the subject matter; it assumes that you've already read a tutorial or a textbook and that you are now seeking to improve your problem-solving skills. You should attempt solving the tasks of the respective kata first, and turn to the workbook only if stuck or for reinforcement. While a textbook emphasizes knowledge acquisition, a workbook emphasizes skill acquisition.

This workbook describes the solutions to the problems offered in the [Ripple Carry Adder Kata](./RippleCarryAdder.ipynb). 
Since the tasks are offered as programming problems, the explanations also cover some elements of Q# that might be non-obvious for a novice.

**What you should know for this workbook**

You should be familiar with the following concepts and associated techniques **prior to** beginning work on the Basic Gates Quantum Kata.

1. [Basic Gates Kata](../BasicGates/BasicGates.ipynb). 
2. [Single-qubit gates](../tutorials/SingleQubitGates/SingleQubitGates.ipynb).
3. List of basic gates available in Q# can be found at [Microsoft.Quantum.Intrinsic](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.intrinsic).
4. [Syntax of flow control statements in Q#](https://docs.microsoft.com/azure/quantum/user-guide/language/statements/iterations) 
5. [Q# conditional branching](https://docs.microsoft.com/azure/quantum/user-guide/language/statements/conditionalbranching) documentation.

You can also consult the [complete Quantum Katas learning path](https://github.com/microsoft/QuantumKatas#learning-path).

## Part I. Simple Adder Outputting to Empty Qubits


### Theory

* [Classical binary adder on Wikipedia](https://en.wikipedia.org/wiki/Adder_(electronics)).
* Part 2 of the [paper on quantum binary addition](https://arxiv.org/pdf/quant-ph/0008033.pdf) by Thomas G. Draper explains how to adapt the classical adder to a quantum environment.

### Task 1.1. Summation of two bits

**Inputs:**

  1. qubit `a` in an arbitrary state $|\phi\rangle$,  
  2. qubit `b` in an arbitrary state $|\psi\rangle$,  
  3. qubit `sum` in state $|0\rangle$.

**Goal:** Transform the `sum` qubit into the lowest bit of the binary sum of $\phi$ and $\psi$.

* $|0\rangle + |0\rangle \to |0\rangle$
* $|0\rangle + |1\rangle \to |1\rangle$
* $|1\rangle + |0\rangle \to |1\rangle$
* $|1\rangle + |1\rangle \to |0\rangle$

Any superposition should map appropriately. 

**Example:** (Recall that $|+\rangle = \frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)$, $|-\rangle = \frac{1}{\sqrt{2}}(|0\rangle - |1\rangle)$)

$|+\rangle \otimes |-\rangle \otimes |0\rangle \to \frac{1}{2}(|000\rangle + |101\rangle - |011\rangle - |110\rangle)$

### Solution

> In this workbook we will reason about the computations acting on basis states, as opposed to arbitrary superposition states. 
> (Since all computations will be implemented using quantum gates, they'll be linear, and behave correctly on superpositions if they are correct on all basis states.) 
> This will allow us to use qubit states and the classical values stored in them interchangeably.

Clearly, the expression we're looking for is $a \oplus b$ - sum of bits $a$ and $b$ modulo 2.

If we apply $CNOT(a,sum)$, then $a=a$, $b=b$, and $sum$ becomes $0 \oplus a = a$.

Again applying $CNOT(b,sum)$, we get $a=a$, $b=b$, and $sum$ becomes $sum \oplus b = a \oplus b$. That's exactly what we're looking for!

In [ ]:
%kata T11_LowestBitSum

operation LowestBitSum (a : Qubit, b : Qubit, sum : Qubit) : Unit is Adj {
    CNOT(a, sum);
    CNOT(b, sum);
}

### Alternate Solution

If we apply $CNOT(a,b)$ , then $a = a$, $b = a \oplus b$, and $sum = 0$.

Again applying $CNOT(b,sum)$, we get $a = a$, $b = a \oplus b$, and $sum = sum \oplus b = a \oplus b$.

Now by applying $CNOT(a,b)$, we get $a = a$, $b = a \oplus (a \oplus b) = b$, $sum = a \oplus b$, and thus we will restore the previous value of $b$.

In [ ]:
%kata T11_LowestBitSum

operation LowestBitSum (a : Qubit, b : Qubit, sum : Qubit) : Unit is Adj {
    CNOT(a, b);
    CNOT(b, sum);
    CNOT(a, b);
}

[Return to Task 1.1 of the Ripple Carry Adder kata.](./RippleCarryAdder.ipynb#Task-1.1.-Summation-of-two-bits)

### Task 1.2. Carry of two bits

**Inputs:**

  1. qubit `a` in an arbitrary state $|\phi\rangle$,
  2. qubit `b` in an arbitrary state $|\psi\rangle$,
  3. qubit `carry` in state $|0\rangle$.

**Goal:** Set the `carry` qubit to $|1\rangle$ if the binary sum of $\phi$ and $\psi$ produces a carry.

* $|0\rangle$ and $|0\rangle \to |0\rangle$
* $|0\rangle$ and $|1\rangle \to |0\rangle$
* $|1\rangle$ and $|0\rangle \to |0\rangle$
* $|1\rangle$ and $|1\rangle \to |1\rangle$

Any superposition should map appropriately. 

**Example:**

$|+\rangle \otimes |-\rangle \otimes |0\rangle \to \frac{1}{2}(|000\rangle + |100\rangle - |010\rangle - |111\rangle)$

### Solution
Clearly, $carry = a \text{ AND } b$.

If we apply $CCNOT(a, b, carry)$, then $a=a$, $b=b$, and $carry$ becomes $(a \text{ AND } b) \oplus carry = (a \text{ AND } b) \oplus 0 = (a \text{ AND } b)$.

In [ ]:
%kata T12_LowestBitCarry

operation LowestBitCarry (a : Qubit, b : Qubit, carry : Qubit) : Unit is Adj {
    CCNOT(a, b, carry);
}

[Return to Task 1.2 of the Ripple Carry Adder kata.](./RippleCarryAdder.ipynb#Task-1.2.-Carry-of-two-bits)

### Task 1.3. One-bit adder

**Inputs:**

  1. qubit `a` in an arbitrary state $|\phi\rangle$,
  2. qubit `b` in an arbitrary state $|\psi\rangle$,
  3. two qubits `sum` and `carry` in state $|0\rangle$.

**Goals:**

* Transform the `sum` qubit into the lowest bit of the binary sum of $\phi$ and $\psi$.
* Transform the `carry` qubit into the carry bit produced by that sum.

### Solution
This task is just a combination of the two previous ones. Using [task 1.1](./Workbook_RippleCarryAdder.ipynb#Task-1.1.-Summation-of-two-bits) and [task 1.2](./Workbook_RippleCarryAdder.ipynb#Task-1.2.-Carry-of-two-bits), 
we can get the solution below.

In [ ]:
%kata T13_OneBitAdder

operation OneBitAdder (a : Qubit, b : Qubit, sum : Qubit, carry : Qubit) : Unit is Adj {
    LowestBitSum(a, b, sum);
    LowestBitCarry(a, b, carry);
}

[Return to Task 1.3 of the Ripple Carry Adder kata.](./RippleCarryAdder.ipynb#Task-1.3.-One-bit-adder)

### Task 1.4. Summation of 3 bits

**Inputs:**

  1. qubit `a` in an arbitrary state $|\phi\rangle$,
  2. qubit `b` in an arbitrary state $|\psi\rangle$,
  3. qubit `carryin` in an arbitrary state $|\omega\rangle$,
  4. qubit `sum` in state $|0\rangle$.

**Goal:** Transform the `sum` qubit into the lowest bit of the binary sum of $\phi$, $\psi$ and $\omega$.

### Solution
Clearly, the expression we're looking for is $a \oplus b \oplus carryin$ - sum of bits $a$ , $b$ and $carryin$ modulo 2.

If we apply $CNOT(a,sum)$ , then $a = a$ , $b = b$ , $carryin = carryin$ and $sum = 0 \oplus a = a$

Again applying $CNOT(b,sum)$, we get $a = a$ , $b = b$ , $carryin = carryin$ and $sum = sum \oplus b = a \oplus b$

Now applying $CNOT(carryin,sum)$, we get $a = a$ , $b = b$ , $carryin = carryin$ and $sum = sum \oplus carryin = a \oplus b \oplus carryin$

In [ ]:
%kata T14_HighBitSum

operation HighBitSum (a : Qubit, b : Qubit, carryin : Qubit, sum : Qubit) : Unit is Adj {
    CNOT(a, sum);
    CNOT(b, sum);
    CNOT(carryin, sum);
}

### Alternate Solution

If we apply $CNOT(a,b)$ , then $a = a$ , $b = a \oplus b$ , $carryin = carryin$ and $sum = 0$

Again applying $CNOT(b,carryin)$, we get $a = a$ , $b = a \oplus b$ , $carryin = carryin \oplus b = carryin \oplus a \oplus b$ and $sum = 0$

Now applying $CNOT(carryin,sum)$, we get $a = a$ , $b = a \oplus b$ , $carryin = carryin \oplus a \oplus b$ and $sum = sum \oplus carryin = a \oplus b \oplus carryin$

Now that we have obtained the sum, we will restore b and carryin to their original values using $CNOT(b,carryin)$ and $CNOT(a,b)$ similar to what we have done in [task 1.1](./Workbook_RippleCarryAdder.ipynb#Task-1.1.-Summation-of-two-bits).

So by applying $CNOT(b,carryin)$, we get $a = a$ , $b = a \oplus b$ , $carryin = carryin \oplus b = (carryin \oplus a \oplus b) \oplus (a \oplus b) = carryin$ and $sum = a \oplus b \oplus carryin$ , we will restore the value of $carryin$

Now by applying $CNOT(a,b)$, we get $a = a$, $b = a \oplus (a \oplus b) = b$, $carryin = carryin$ and $sum = a \oplus b \oplus carryin$, thus we will restore the previous value of $b$.

In [ ]:
%kata T14_HighBitSum

operation HighBitSum (a : Qubit, b : Qubit, carryin : Qubit, sum : Qubit) : Unit is Adj {
    CNOT(a, b);
    CNOT(b, carryin);
    CNOT(carryin, sum);
    CNOT(b, carryin);
    CNOT(a, b);
}

[Return to Task 1.4 of the Ripple Carry Adder kata.](./RippleCarryAdder.ipynb#Task-1.4.-Summation-of-3-bits)

### Task 1.5. Carry of 3 bits

**Inputs:**

  1. qubit `a` in an arbitrary state $|\phi\rangle$,
  2. qubit `b` in an arbitrary state $|\psi\rangle$,
  3. qubit `carryin` in an arbitrary state $|\omega\rangle$,
  4. qubit `carryout` in state $|0\rangle$.

**Goal:** Transform the `carryout` qubit into the carry bit produced by the sum of $\phi$, $\psi$ and $\omega$.

### Solution
Clearly, $carryout$ = $(a \text{ AND } b) \oplus (b \text{ AND } carryin) \oplus (carryin \text{ AND } a)$
 as carryout is in state |1> only when atleast 2 of a, b, carryin are in state |1>.

This means that either $1$ or $3$ of $(a \text{ AND } b), (b \text{ AND } carryin) and (carryin \text{ AND } a)$ are in state $|1>$ 
i.e $carryout = (a \text{ AND } b) \oplus (b \text{ AND } carryin) \oplus (carryin \text{ AND } a)$


Now if we apply $CCNOT(a,b,carryout)$ , then $a = a$, $b = b$, $carryin = carryin$ and $carryout = (a \text{ AND } b) \oplus carryout = (a \text{ AND } b) \oplus 0 = (a \text{ AND } b)$

Again by applying $CCNOT(b,carryin,carryout)$ , $a = a$, $b = b$, $carryin = carryin$ and $carryout = (b \text{ AND } carryin) \oplus carryout = (b \text{ AND } carryin) \oplus (a \text{ AND } b)$

Finally by applying $CCNOT(carryin,a,carryout)$ , $a = a$, $b = b$, $carryin = carryin$ and $carryout = (carryin \text{ AND } a) \oplus carryout = (carryin \text{ AND } a) \oplus (b \text{ AND } carryin) \oplus (a \text{ AND } b)$

In [ ]:
%kata T15_HighBitCarry

operation HighBitCarry (a : Qubit, b : Qubit, carryin : Qubit, carryout : Qubit) : Unit is Adj {
    CCNOT(a, b, carryout);
    CCNOT(b, carryin, carryout);
    CCNOT(carryin, a, carryout);
}

[Return to Task 1.5 of the Ripple Carry Adder kata.](./RippleCarryAdder.ipynb#Task-1.5.-Carry-of-3-bits)

### Task 1.6. Two-bit adder

**Inputs:**

  1. two-qubit register `a` in an arbitrary state $|\phi\rangle$,
  2. two-qubit register `b` in an arbitrary state $|\psi\rangle$,
  3. two-qubit register `sum` in state $|00\rangle$,
  4. qubit `carry` in state $|0\rangle$.

**Goals:**

* Transform the `sum` register into the binary sum (little-endian) of $\phi$ and $\psi$.
* Transform the `carry` qubit into the carry bit produced by that sum.

> All registers in this kata are in **little-endian** order.
> This means that they have the least significant bit first, followed by the next least significant, and so on.
>
> In this exercise, for example, $1$ would be represented as $|10\rangle$, and $2$ as $|01\rangle$.
>
> The sum of $|10\rangle$ and $|11\rangle$ would be $|001\rangle$, with the last qubit being the carry qubit.

<br/>
<details>
    <summary><b>Need a hint? Click here</b></summary>
    Don't forget that you can allocate extra qubits.
</details>

### Solution

First we will define a qubit for $carryin$.

The use the $OneBitAdder$ function made in [task 1.3](./Workbook_RippleCarryAdder.ipynb#Task-1.3.-One-bit-adder) for getting the $sum$ of the $lsb$(least significant bit) i.e. $a[0]$ and $b[0]$ as $sum[0]$ and the $carry$ as $carry in$. 

Next we will get the $sum$ and $carry$ for the $msb$(most significant bit) i.e. $a[1]$ and $b[1]$ by using $HighBitSum$ in [task 1.4](./Workbook_RippleCarryAdder.ipynb#Task-1.4.-Summation-of-3-bits) and $HighBitCarry$ in [task 1.5](./Workbook_RippleCarryAdder.ipynb#Task-1.4.-Carry-of-3-bits).

At the end we will apply $CCNOT(a[0],b[0],carryin)$ to restore the value of $carryin$ (as $carryin$ was $a[0] \text{ AND } b[0]$).

In [ ]:
%kata T16_TwoBitAdder

operation TwoBitAdder (a : Qubit[], b : Qubit[], sum : Qubit[], carry : Qubit) : Unit is Adj {
    use carryin = Qubit();
    
    OneBitAdder(a[0], b[0], sum[0], carryin);
    HighBitSum(a[1], b[1], carryin, sum[1]);
    HighBitCarry(a[1], b[1], carryin, carry);
    
    CCNOT(a[0],b[0],carryin);
}

[Return to Task 1.6 of the Ripple Carry Adder kata.](./RippleCarryAdder.ipynb#Task-1.6.-Two-bit-adder|)

### Task 1.7. N-bit adder

**Inputs:**

  1. $N$-qubit register `a` in an arbitrary state $|\phi\rangle$,
  2. $N$-qubit register `b` in an arbitrary state $|\psi\rangle$,
  3. $N$-qubit register `sum` in state $|0...0\rangle$,
  4. qubit `carry` in state $|0\rangle$.

**Goals:**

* Transform the `sum` register into the binary sum (little-endian) of $\phi$ and $\psi$.
* Transform the `carry` qubit into the carry bit produced by that sum.

**Challenge:**

Can you do this without allocating extra qubits?

### Solution

First we will get the length n of the n-bit adder and define a qubit of length n for carryin.

Then we can use the $OneBitAdder$ function made in [task 1.3](./Workbook_RippleCarryAdder.ipynb#Task-1.3.-One-bit-adder) for getting the sum of the lsb(least significant bits) as $sum[0]$ and the carry as carry in. 

Next we will make a loop to get the sum using $HighBitSum$ in [task 1.4](./Workbook_RippleCarryAdder.ipynb#Task-1.4.-Summation-of-3-bits) and carryin(as internal carry) using $HighBitCarry$ in [task 1.5](./Workbook_RippleCarryAdder.ipynb#Task-1.4.-Carry-of-3-bits) till we reach the MSB(maximum significant bit).

After this we will apply $CNOT(carryin[n-1],carry)$ , to get carry as the final carryin that we have.

Now that we have the required $sum$ and $carry$, we have to apply a reverse for loop to get the $carryin[i]$ for all ellements in $carryin$ except $carryin[0]$ to get the released qubits in zero state.

At the end we will apply $CCNOT(a[0],b[0],carryin[0])$ to restore the value of $carryin[0]$ as 0 (as $carryin$ was $a[0] \text{ AND } b[0]$).

In [ ]:
%kata T17_ArbitraryAdder

operation ArbitraryAdder (a : Qubit[], b : Qubit[], sum : Qubit[], carry : Qubit) : Unit is Adj {
    let n = Length(a);
    use carryin = Qubit[n];
    
    OneBitAdder(a[0],b[0],sum[0],carryin[0]);
    for i in 1..n-1{
        HighBitSum(a[i],b[i],carryin[i-1],sum[i]);
        HighBitCarry(a[i],b[i],carryin[i-1],carryin[i]);
    }
    CNOT(carryin[n-1],carry);
    
    for i in n-1 .. -1 .. 1 {
        Adjoint HighBitCarry(a[i], b[i], carryin[i-1], carryin[i]);
    }
    CCNOT(a[0], b[0], carryin[0]);
}

### Challenge Solution

As we are trying to solve this task without using additional qubits, we will store the $carryin$ used in the above solution in sum (which is initially empty).

First we will get the length n of the n-bit adder.

Then we can use the $LowestBitCarry$ function made in [task 1.2](./Workbook_RippleCarryAdder.ipynb#Task-1.2.-Carry-of-two-bits) to get the carry of the lsb(least significant bits) as $sum[0]$. 

Now we will make a loop from i = 1 to n and use the $HighBitCarry(a[i], b[i], sum[i - 1], sum[i])$ using $sum[i-1]$ as $carryin$ and $sum[i]$ as $carryout$. See [task 1.5](./Workbook_RippleCarryAdder.ipynb#Task-1.4.-Carry-of-3-bits).

Then we will apply $CNOT(sum[N-1], carry)$ to get the final $carry$


Now that we have obtained the required carry, we will work on getting the sum part

For this we will make a loop from n-1 to 1 by using $Adjoint HighBitCarry(a[i], b[i], sum[i - 1], sum[i])$ to restore the value of $sum[i]$ as 0 (as we already utilized the carryin value stored in $sum[i]$) and then use $HighBitSum$ function made in [task 1.4](./Workbook_RippleCarryAdder.ipynb#Task-1.4.-Summation-of-3-bits) to get the value of $sum[i]$. Note that $sum[i-1]$ still contains the respective $carryin$ which we are using to get $sum[i]$.

Now we will apply $Adjoint LowestBitCarry(a[0], b[0], sum[0])$ to restore the value of $sum[0]$ and then use the $LowestBitSum$ function made in [task 1.1](./Workbook_RippleCarryAdder.ipynb#Task-1.1.-Summation-of-two-bits) to get the correct value of $sum[0]$.

In [ ]:
%kata T17_ArbitraryAdder

operation ArbitraryAdder (a : Qubit[], b : Qubit[], sum : Qubit[], carry : Qubit) : Unit is Adj {
    let N = Length(a);

    LowestBitCarry(a[0], b[0], sum[0]);
    for i in 1 .. N-1 {
        HighBitCarry(a[i], b[i], sum[i - 1], sum[i]);
    }
    CNOT(sum[N-1], carry);

    for i in N-1 .. -1 .. 1 {
        Adjoint HighBitCarry(a[i], b[i], sum[i - 1], sum[i]);
        HighBitSum(a[i], b[i], sum[i - 1], sum[i]);
    }
    Adjoint LowestBitCarry(a[0], b[0], sum[0]);
    LowestBitSum(a[0], b[0], sum[0]);
}

[Return to Task 1.7 of the Ripple Carry Adder kata.](./RippleCarryAdder.ipynb#Task-1.7.-N-bit-adder)

More solutions comming soon..